In [36]:
import pandas as pd
import numpy as np

data = pd.read_csv("police2.csv")

## 데이터 분포 확인

In [54]:
df = data

In [38]:
df.keys()

Index(['Unnamed: 0', 'NPA_CL', 'EVT_STAT_CD', 'EVT_CL_CD', 'HPPN_PNU_ADDR',
       'SME_EVT_YN', '1', '2', '3', 'RECV_CPLT_DM', 'year', 'month', 'hour',
       '대전', '세종', '충남'],
      dtype='object')

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2644824 entries, 0 to 2644823
Data columns (total 16 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   Unnamed: 0     int64 
 1   NPA_CL         int64 
 2   EVT_STAT_CD    int64 
 3   EVT_CL_CD      int64 
 4   HPPN_PNU_ADDR  object
 5   SME_EVT_YN     object
 6   1              int64 
 7   2              int64 
 8   3              int64 
 9   RECV_CPLT_DM   object
 10  year           int64 
 11  month          int64 
 12  hour           int64 
 13  대전             int64 
 14  세종             int64 
 15  충남             int64 
dtypes: int64(13), object(3)
memory usage: 322.9+ MB


In [40]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df

,NPA_CL,EVT_STAT_CD,EVT_CL_CD,HPPN_PNU_ADDR,SME_EVT_YN,1,2,3,RECV_CPLT_DM,year,month,hour,대전,세종,충남
0,19,10,0,충청남도 보령시 궁촌동(행정:대천4동) 369,Y,1,0,0,2020-12-01 01:43:07,2020,12,1,0,0,1
1,13,10,0,0,0,0,0,1,2020-12-01 02:05:04,2020,12,2,1,0,0
2,13,10,0,0,0,1,0,0,2020-12-01 02:06:52,2020,12,2,1,0,0
3,19,10,0,충청남도 보령시 천북면 하만리 628-10,N,0,0,1,2020-12-01 02:37:25,2020,12,2,0,0,1
4,19,10,0,충청남도 천안시 서북구 성정동(행정:성정2동) 1259,0,0,1,0,2020-12-01 08:17:50,2020,12,8,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2644819,13,10,0,대전광역시 유성구 봉명동 (행정:원신흥동 ) 1016-7,0,1,0,0,2023-01-12 11:23:09,2023,1,11,1,0,0
2644820,19,10,0,충청남도 서산시 음암면 부장리(음암면 ) 560-4,0,0,1,0,2023-01-16 11:59:47,2023,1,11,0,0,1
2644821,19,10,0,0,0,0,0,1,2023-01-16 11:58:54,2023,1,11,0,0,1
2644822,19,10,0,0,0,0,0,1,2023-01-14 01:00:47,2023,1,1,0,0,1


## library

In [41]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE

from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score # 정확도 함수
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance

In [42]:
os.getcwd() # 현재 위치 확인
path = os.getcwd() # 현재 위치를 path 변수에 저장
os.chdir(path) # 현재 위치 변경

In [55]:
encoder =  preprocessing.LabelEncoder() 
df['EVT_CL_CD'] = encoder.fit_transform(df['EVT_CL_CD'])
df['EVT_CL_CD'] = df['EVT_CL_CD'].astype('float32')

In [56]:
df = df[['year','대전', '세종', '충남', 'EVT_CL_CD']]
df

,year,대전,세종,충남,EVT_CL_CD
0,2020,0,0,1,0.0
1,2020,1,0,0,0.0
2,2020,1,0,0,0.0
3,2020,0,0,1,0.0
4,2020,0,0,1,0.0
...,...,...,...,...,...
2644819,2023,1,0,0,0.0
2644820,2023,0,0,1,0.0
2644821,2023,0,0,1,0.0
2644822,2023,0,0,1,0.0


## 데이터 분리

In [57]:
x = df.iloc[:, :4].values
y = df.iloc[:, -1:].values

y = y.ravel()
print(x.shape, y.shape)

(2644824, 4) (2644824,)


In [58]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [60]:
train_set_feature = pd.DataFrame(x_train, columns=['year','대전', '세종', '충남'])
train_set_label = pd.DataFrame(y_train, columns=['EVT_CL_CD'])
train_set = pd.concat([train_set_feature, train_set_label], axis = 1)

test_set_feature = pd.DataFrame(x_test, columns=['year','대전', '세종', '충남'])
test_set_label = pd.DataFrame(y_test, columns=['EVT_CL_CD'])
test_set = pd.concat([test_set_feature, test_set_label], axis = 1)

## Smote

In [61]:
scaler = preprocessing.MinMaxScaler()

train_sc = scaler.fit_transform(train_set)
test_sc = scaler.transform(test_set)

## train data split

In [62]:
# train
x_train_values = train_sc[:, :-1] # 독립변수 분리
y_train_values = train_sc[:,-1] # 종속변수 분리

# test
x_test_values = test_sc[:, :-1] # 독립변수 분리
y_test_values = test_sc[:,-1] # 종속변수 분리

In [63]:
smote = SMOTE()
x_train_over, y_train_over = smote.fit_resample(x_train_values, y_train_values)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ', x_train_values.shape, y_train_values.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : ', x_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (1851376, 4) (1851376,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :  (3652992, 4) (3652992,)
SMOTE 적용 후 레이블 값 분포: 
 0.0    1826496
1.0    1826496
dtype: int64


# Logistic Regression

In [64]:
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression()
lrc.fit(x_train_over, y_train_over)

LogisticRegression()

In [65]:
predict3 = lrc.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict3))
print('f1_score :', f1_score(y_test_values, predict3))
print('precision : ', precision_score(y_test_values, predict3))
print('recall :', recall_score(y_test_values, predict3))

accuracy_score : 0.4117371270707091
f1_score : 0.03429926303549926
precision :  0.017534507377439314
recall : 0.781317749080969


In [66]:
confusion_matrix(y_test_values, predict3)

array([[318403, 464436],
       [  2320,   8289]], dtype=int64)

# Decision Tree

In [67]:
from sklearn.tree import DecisionTreeClassifier

dsc = DecisionTreeClassifier()
dsc.fit(x_train_over, y_train_over)

DecisionTreeClassifier()

In [68]:
predict1 = dsc.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict1))
print('f1_score :', f1_score(y_test_values, predict1))
print('precision : ', precision_score(y_test_values, predict1))
print('recall :', recall_score(y_test_values, predict1))

accuracy_score : 0.4123193958520281
f1_score : 0.03442806261906735
precision :  0.017600691070078826
recall : 0.78357997926289


In [69]:
confusion_matrix(y_test, predict1)

array([[318841, 463998],
       [  2296,   8313]], dtype=int64)

# Random Forest

In [75]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(x_train_over,y_train_over)

RandomForestClassifier()

In [76]:
predict2 = clf.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict2))
print('f1_score :', f1_score(y_test_values, predict2))
print('precision : ', precision_score(y_test_values, predict2))
print('recall :', recall_score(y_test_values, predict2))

accuracy_score : 0.4123193958520281
f1_score : 0.03442806261906735
precision :  0.017600691070078826
recall : 0.78357997926289


In [77]:
confusion_matrix(y_test, predict1)

array([[318841, 463998],
       [  2296,   8313]], dtype=int64)

# 변수 추가해서 넣기

In [82]:
df2 = data[['year','1','2','3','대전', '세종', '충남', 'EVT_CL_CD']]
df2

,year,1,2,3,대전,세종,충남,EVT_CL_CD
0,2020,1,0,0,0,0,1,0.0
1,2020,0,0,1,1,0,0,0.0
2,2020,1,0,0,1,0,0,0.0
3,2020,0,0,1,0,0,1,0.0
4,2020,0,1,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...
2644819,2023,1,0,0,1,0,0,0.0
2644820,2023,0,1,0,0,0,1,0.0
2644821,2023,0,0,1,0,0,1,0.0
2644822,2023,0,0,1,0,0,1,0.0


## 데이터 분리

In [83]:
x = df2.iloc[:, :7].values
y = df2.iloc[:, -1:].values

y = y.ravel()
print(x.shape, y.shape)

(2644824, 7) (2644824,)


In [84]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2)

train_set_feature = pd.DataFrame(x_train, columns=['year','1','2','3','대전', '세종', '충남'])
train_set_label = pd.DataFrame(y_train, columns=['EVT_CL_CD'])
train_set = pd.concat([train_set_feature, train_set_label], axis = 1)

test_set_feature = pd.DataFrame(x_test, columns=['year','1','2','3','대전', '세종', '충남'])
test_set_label = pd.DataFrame(y_test, columns=['EVT_CL_CD'])
test_set = pd.concat([test_set_feature, test_set_label], axis = 1)

## Smote

In [85]:
scaler = preprocessing.MinMaxScaler()

train_sc = scaler.fit_transform(train_set)
test_sc = scaler.transform(test_set)

## train data split

In [87]:
# train
x_train_values = train_sc[:, :-1] # 독립변수 분리
y_train_values = train_sc[:,-1] # 종속변수 분리

# test
x_test_values = test_sc[:, :-1] # 독립변수 분리
y_test_values = test_sc[:,-1] # 종속변수 분리

smote = SMOTE()
x_train_over, y_train_over = smote.fit_resample(x_train_values, y_train_values)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트 : ', x_train_values.shape, y_train_values.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트 : ', x_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트 :  (1851376, 7) (1851376,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트 :  (3653064, 7) (3653064,)
SMOTE 적용 후 레이블 값 분포: 
 0.0    1826532
1.0    1826532
dtype: int64


# Logistic Regression

In [89]:
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression()
lrc.fit(x_train_over, y_train_over)

LogisticRegression()

In [90]:
predict3 = lrc.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict3))
print('f1_score :', f1_score(y_test_values, predict3))
print('precision : ', precision_score(y_test_values, predict3))
print('recall :', recall_score(y_test_values, predict3))

accuracy_score : 0.6198515844768655
f1_score : 0.04657322940176128
precision :  0.024097449602082974
recall : 0.6920620009394082


In [105]:
confusion_matrix(y_test, predict3)

array([[484453, 298350],
       [  3278,   7367]], dtype=int64)

# Decision Tree

In [91]:
from sklearn.tree import DecisionTreeClassifier

dsc = DecisionTreeClassifier()
dsc.fit(x_train_over, y_train_over)

predict1 = dsc.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict1))
print('f1_score :', f1_score(y_test_values, predict1))
print('precision : ', precision_score(y_test_values, predict1))
print('recall :', recall_score(y_test_values, predict1))

accuracy_score : 0.620101128240288
f1_score : 0.04659636515457266
precision :  0.024109951328403986
recall : 0.6919680601221231


In [92]:
confusion_matrix(y_test, predict1)

array([[484652, 298151],
       [  3279,   7366]], dtype=int64)

# Random Forest

In [103]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
clf.fit(x_train_over,y_train_over)

predict2 = clf.predict(x_test_values)
print('accuracy_score :', accuracy_score(y_test_values ,predict2))
print('f1_score :', f1_score(y_test_values, predict2))
print('precision : ', precision_score(y_test_values, predict2))
print('recall :', recall_score(y_test_values, predict2))

accuracy_score : 0.620101128240288
f1_score : 0.04659636515457266
precision :  0.024109951328403986
recall : 0.6919680601221231


In [104]:
confusion_matrix(y_test, predict2)

array([[484652, 298151],
       [  3279,   7366]], dtype=int64)